In [77]:
!pip install pymongo

In [152]:
from pymongo import MongoClient
from tabulate import tabulate
import datetime
from haversine import haversine
from collections import Counter
# Connect to the MongoDB server running on localhost at the default port 27017
client = MongoClient('mongodb://localhost:27017/')


In [79]:
# Create or access a database called 'mydatabase'
db = client['test_db']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test_db')

In [80]:
user = db["user"]
activity = db["activity"]
trackpoint = db["trackpoint"]

### 1. How many users, activities and trackpoints are there in the dataset (after it is inserted into the database).

In [81]:
user_count = user.count_documents({})
activity_count = activity.count_documents({})
trackpoint_count = trackpoint.count_documents({})
print("Number of users:",user_count, ", Number of activities:", activity_count, ", Number of trackpoints:",trackpoint_count)

Number of users: 182 , Number of activities: 16030 , Number of trackpoints: 9636851


### 2. Find the average number of activities per user

In [82]:

pipeline = [
    {
        "$group": {
            "_id": "$user_id",
            "count": {"$sum": 1}
        }
    },
    {
        "$group": {
            "_id": None,
            "average_activities_per_user": {"$avg": "$count"}
        }
    }
]

result = activity.aggregate(pipeline)
average = next(result, None)["average_activities_per_user"]
average


92.65895953757226

### 3. Find the top 20 users with the highest number of activities. 

In [83]:
pipeline = [
    {
        "$group": {
            "_id": "$user_id",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 20
    }
]

result3 = activity.aggregate(pipeline)
print(tabulate(result3, headers="keys", tablefmt="pretty"))

+-----+-------+
| _id | count |
+-----+-------+
| 128 | 2101  |
| 153 | 1791  |
| 025 |  715  |
| 163 |  704  |
| 062 |  691  |
| 144 |  563  |
| 041 |  398  |
| 085 |  364  |
| 004 |  346  |
| 140 |  345  |
| 167 |  320  |
| 068 |  278  |
| 017 |  265  |
| 003 |  261  |
| 014 |  232  |
| 126 |  215  |
| 030 |  210  |
| 112 |  208  |
| 011 |  201  |
| 039 |  198  |
+-----+-------+


### 4. Find all users who have taken a taxi. 

In [84]:
pipeline = [
    {
        "$match": {"transportation_mode": "taxi"}
    },
    {
        "$group": {
            "_id": "$user_id",
        }
    }
]

result4 = activity.aggregate(pipeline)
print(tabulate(result4, headers="keys", tablefmt="pretty"))

+-----+
| _id |
+-----+
| 098 |
| 128 |
| 163 |
| 058 |
| 010 |
| 062 |
| 078 |
| 080 |
| 111 |
| 085 |
+-----+


### 5. Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null.

In [85]:
pipeline = [
    {
        "$match": {"transportation_mode": {"$ne": None}}
    },
    {
        "$group": {
            "_id": "$transportation_mode",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    }
]

result5 = activity.aggregate(pipeline)
print(tabulate(result5, headers="keys", tablefmt="pretty"))


+----------+-------+
|   _id    | count |
+----------+-------+
|   walk   |  481  |
|   car    |  419  |
|   bike   |  262  |
|   bus    |  199  |
|  subway  |  133  |
|   taxi   |  37   |
| airplane |   3   |
|  train   |   2   |
|   boat   |   1   |
|   run    |   1   |
+----------+-------+


### 6. a) Find the year with the most activities. 
We assume that if an activity starts in a year, it is to be conted as in that year. In practice, this changes nothing, but there is a teoretical chance of someone starting an activity in one year, and ending it in the next.

In [86]:
pipeline = [
    {
        "$project": {
            "year": {"$year": "$start_date_time"}
        }
    },
    {
        "$group": {
            "_id": "$year",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 1
    }
]

result6a = activity.aggregate(pipeline)
year_most = next(result6a,None)["_id"]
print("The year with the most activities was", year_most)

The year with the most activities was 2008


### b) Is this also the year with most recorded hours?
Sorting them by total time, we see that 2009 had more total time. To conclude 2008 did not have the most logged hours.

In [87]:
pipeline = [
    {
        "$project": {
            "year": {"$year": "$start_date_time"},
            "duration": {
                "$subtract": ["$end_date_time", "$start_date_time"]
            }
        }
    },
    {
        "$group": {
            "_id": "$year",
            "count": {"$sum": 1},
            "total_time": {"$sum": "$duration"}
        }
    },
    {
        "$sort": {"total_time": -1}
    },
    {
        "$project": {
            "_id": 1
        }
    }
]

result6b = activity.aggregate(pipeline)
print(tabulate(result6b, headers="keys", tablefmt="pretty"))


+------+
| _id  |
+------+
| 2009 |
| 2008 |
| 2007 |
| 2010 |
| 2011 |
| 2012 |
| 2000 |
+------+


### 7. Find the total distance (in km) walked in 2008, by user with id=112.

In [88]:


activities_found = activity.find({
            "user_id": {"$eq": "112"},
            "transportation_mode": {"$eq": "walk"},
            "start_date_time": {"$gte": datetime.datetime(2008, 1, 1), "$lt": datetime.datetime(2009, 1, 1)},
            })

activity_ids = [act["_id"] for act in activities_found] 


In [94]:
total_distance = 0.0

for activity_id in activity_ids:
    trackpoints = list(trackpoint.find({"activity_id": {"$eq": activity_id}}))
    for i in range(1, len(trackpoints)):
        lat1, lon1 = trackpoints[i-1]['lat'], trackpoints[i-1]['lon']
        lat2, lon2 = trackpoints[i]['lat'], trackpoints[i]['lon']
        total_distance += haversine((lat1, lon1), (lat2, lon2))


print("The total distance in km is:",total_distance)

The total distance in km is: 115.47465961507991


### 8. Find the top 20 users who have gained the most altitude meters.

In [95]:
all_activities = activity.find({},{"_id": 1})
all_activity_ids = [act["_id"] for act in all_activities] 

activity_gain = {}
for activity_id in all_activity_ids:
    activity_gain[activity_id] = 0
    trackpoints = list(trackpoint.find({"activity_id": {"$eq": activity_id}}))
    for i in range(1, len(trackpoints)):
        difference = trackpoints[i]['altitude'] - trackpoints[i-1]['altitude']
        if difference > 0: activity_gain[activity_id] += difference
        

In [96]:
top20 = sorted(activity_gain.items(), key= lambda x: x[1],reverse = True)[:20]

print(tabulate(top20, headers=["user_id","total gain (m)"], tablefmt="pretty"))

+---------+--------------------+
| user_id |   total gain (m)   |
+---------+--------------------+
|  4426   |      107752.5      |
|  10484  |      107493.3      |
|  4578   |      79724.5       |
|  4816   | 72030.70000000003  |
|  11272  | 68936.70000000003  |
|  11682  | 55823.399999999994 |
|  8438   | 55652.887139107675 |
|  5529   | 52905.70000000005  |
|  4872   | 49425.899999999994 |
|  4452   | 49330.500000000015 |
|  4117   | 46591.60000000003  |
|  12296  |      46207.3       |
|  9734   | 44904.99999999999  |
|  7892   |      44645.6       |
|  9743   |      43139.7       |
|  12288  | 43038.20000000004  |
|  9922   | 42850.99999999998  |
|  9934   | 42414.099999999904 |
|  9726   | 42136.40000000003  |
|  10980  | 41863.899999999965 |
+---------+--------------------+


### 

### 9. Find all users who have invalid activities, and the number of invalid activities per user

In [175]:
pipeline = [
    {"$sort": {"date": 1}}
]
trackpoints_sorted = trackpoint.aggregate(pipeline)

illegal_activities = set()
prev_point = None
for point in trackpoints_sorted:
    if prev_point == None: 
        prev_point = point
        continue
    timediff = (point["date"] - prev_point["date"]).total_seconds()
    if timediff > 60 * 5 and point["activity_id"] == prev_point["activity_id"]:  # 5 minuttes
        illegal_activities.add(point["activity_id"])
    prev_point = point


In [190]:
pipeline = [
    {
        "$match": {
            "_id": {"$in": list(illegal_activities)}
        }
    },
    {
        "$group": {
            "_id": "$user_id",
            "count": {"$sum": 1}
        }
    }
]

result9 = activity.aggregate(pipeline)
print(tabulate(result9, headers="keys", tablefmt="pretty"))

+-----+-------+
| _id | count |
+-----+-------+
| 010 |  25   |
| 111 |  22   |
| 173 |   2   |
| 035 |  11   |
| 067 |  31   |
| 028 |   6   |
| 121 |   4   |
| 142 |  35   |
| 119 |  13   |
| 002 |  21   |
| 086 |   4   |
| 131 |   6   |
| 176 |   5   |
| 107 |   1   |
| 096 |   8   |
| 036 |  10   |
| 164 |   2   |
| 072 |   2   |
| 138 |  10   |
| 168 |  12   |
| 030 |  36   |
| 078 |  11   |
| 062 |  132  |
| 146 |   1   |
| 026 |   4   |
| 008 |   3   |
| 022 |  21   |
| 003 |  25   |
| 000 |   5   |
| 084 |  25   |
| 016 |   5   |
| 063 |   8   |
| 043 |   5   |
| 033 |   1   |
| 015 |   4   |
| 112 |  43   |
| 040 |   2   |
| 034 |  22   |
| 082 |  14   |
| 152 |   1   |
| 179 |   5   |
| 042 |  13   |
| 144 |  61   |
| 041 |  57   |
| 013 |   8   |
| 090 |   3   |
| 098 |   5   |
| 069 |   3   |
| 032 |   3   |
| 147 |  30   |
| 105 |   6   |
| 068 |  43   |
| 020 |  17   |
| 081 |   2   |
| 060 |   1   |
| 038 |  15   |
| 061 |   6   |
| 091 |  55   |
| 099 |  11   |
| 056 | 

### 10.Find the users who have tracked an activity in the Forbidden City of Beijing.

In [120]:
beijing_lat = 39.916
beijing_lon = 116.397
buffer = 0.0005

pipeline = [
    {
        "$match": {
            "lat": {"$gte": beijing_lat - buffer, "$lte": beijing_lat + buffer},
            "lon": {"$gte": beijing_lon - buffer, "$lte": beijing_lon + buffer}
        }
    },
    {
        "$group": {
            "_id": "$activity_id"
        }
    },
    {
        "$lookup": {
            "from": "activity",
            "localField": "_id",
            "foreignField": "_id",
            "as": "activity_info"
        }
    },
    {
        "$unwind": "$activity_info"
    },
    {
        "$replaceRoot": { "newRoot": "$activity_info" }
    },
    {
        "$group": {
            "_id": "$user_id",
            "number_of_illegal": {"$sum": 1}
        }
    }
]

result10 = trackpoint.aggregate(pipeline)

print(tabulate(result10, headers="keys", tablefmt="pretty"))

+-----+-------------------+
| _id | number_of_illegal |
+-----+-------------------+
| 004 |         1         |
| 018 |         3         |
| 131 |         1         |
+-----+-------------------+


### 11.Find all users who have registered transportation_mode and their most used transportation_mode. 

In [148]:
pipeline = [
    {
        "$match": {"transportation_mode": {"$ne": None}}
    },
    {
        "$group": {
            "_id": {
                "user_id": "$user_id",
                "transportation_mode": "$transportation_mode"
            },
            "count": {"$sum": 1}  
        }
    },
    {
        "$sort": {
            "_id.user_id": 1,
            "count": -1 
        }
    },
    {
        "$group": {
            "_id": "$_id.user_id",
            "most_used": {"$first": "$_id.transportation_mode"}
        }
    },
    {
        "$sort": {
            "_id": 1 
        }
    }
]

result11 = activity.aggregate(pipeline)
print(tabulate(result11, headers="keys", tablefmt="pretty"))

+-----+-----------+
| _id | most_used |
+-----+-----------+
| 010 |   taxi    |
| 020 |   bike    |
| 021 |   walk    |
| 052 |    bus    |
| 056 |   bike    |
| 058 |    car    |
| 060 |   walk    |
| 062 |    bus    |
| 064 |   bike    |
| 065 |   bike    |
| 067 |   walk    |
| 069 |   bike    |
| 073 |   walk    |
| 075 |   walk    |
| 076 |    car    |
| 078 |   walk    |
| 080 |   taxi    |
| 081 |   bike    |
| 082 |   walk    |
| 084 |   walk    |
| 085 |   walk    |
| 086 |    car    |
| 087 |   walk    |
| 089 |    car    |
| 091 |   walk    |
| 092 |   walk    |
| 097 |   bike    |
| 098 |   taxi    |
| 101 |    car    |
| 102 |   bike    |
| 107 |   walk    |
| 108 |   walk    |
| 111 |   taxi    |
| 112 |   walk    |
| 115 |    car    |
| 117 |   walk    |
| 125 |   bike    |
| 126 |   bike    |
| 128 |    car    |
| 136 |   walk    |
| 138 |   bike    |
| 139 |   bike    |
| 144 |   walk    |
| 153 |   walk    |
| 161 |   walk    |
| 163 |   bike    |
| 167 |   bike    |
